# Qiskit Notebook 6 - Qiskit Primitives


In this Notebook, we will learn about:

- Parametric Circuits
- Observables and Expectation Values
- Estimator and Sampler Primitives

In [ ]:
# If using Google Colab, uncomment the following

#!pip install -q qiskit
#!pip install -q qiskit[visualization]
#!pip install -q qiskit-ibm-runtime
#!pip install -q qiskit-aer

# Qiskit Primitives

A **primitive** is the smallest processing instruction (i.e. simplest building block) from which one can obtain a useful result for a given abstraction level. Abstractions allow you to focus on a particular level of detail relevant for a given task: the closer to the hardware, the lower the level of abstraction. Similarly, the more complex the task, the higher the abstraction.


Qiskit has two primitives:
- The `Sampler` primitive, which corresponds to a **low-level** execution of circuits, samples the output register from quantum circuit execution. In other words, it performs measurements and computes probabilities according to the number of experiments (shots).

- The `Estimator` primitive, which corresponds to a **high-level** execution for algorithms, computes expectation values of observables with respect to states prepared by quantum circuits.


Before studying in detail the inputs/outputs of the `Sampler` and `Estimator`, lets consider a simple example using the reference (noiseless) primitives in `qiskit.primitives`. For this, we will discuss first **parametric circuits** and **expectation values of observables**.

## Parametric Circuits

First, we will introduce **parametric quantum circuits**, that is, quantum circuits that depend on a parameter, which is to be defined later during the execution. To do this, we first construct the circuit, using the `Parameter` class:

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

phi = Parameter('φ')                      # We can use Unicode strings!

qc_param = QuantumCircuit(2)
qc_param.ry(phi, 0)                       # Instead of specifying the value of theta, Parameter acts as a "placeholder"
qc_param.cx(0,1)
qc_param.measure_all()

print(qc_param.parameters)                # Get parameters
qc_param.draw("mpl")

The value of a `Parameter` must be entirely determined before a circuit begins execution. Typically this will mean that you should supply values for every `Parameter` in a circuit using the method `QuantumCircuit.assign_parameters()`

In [ ]:
bc = qc_param.assign_parameters({phi: np.pi/2})
bc.draw('mpl')

The `StatevectorSampler` class takes circuits and parameters as inputs and returns the results from sampling from the output probability distributions. The Sampler is executed similarly to the simulations that we did through a backend, that is, using `sampler.run()` instead of `backend.run()`.

In [ ]:
from qiskit.primitives import StatevectorSampler

sampler = StatevectorSampler()                                     # create sampler class

job_sampler1 = sampler.run([(qc_param, np.pi/2)], shots = 100)     # run the circuit with the parameter value using the sampler
data_sampler = job_sampler1.result()[0].data                       # get data from the result
counts1 = data_sampler.meas.get_counts()                           # get the counts
counts1

## Observables and Expectation Values

In quantum mechanics, **observables** correspond to physical properties that can be measured. For our purposes, an observable is a Hermitian operator (matrix), that is, an operator that satisfies $$O^{\dagger} = O.$$ A common example, which will be explored in more detail later on, is the **Hamiltonian**, which represents the total energy of the quantum system. To measure an $n$-qubit observable $O$ on a quantum computer, we must represent it as a sum of tensor products of **Pauli operators**, that is,

$$
O=\sum_{k=1}^{K} \alpha_{k} P_{k}, \, \, \, P_{k} \in\{I, X, Y, Z \}^{\otimes n}, \, \, \, \alpha_{k} \in\mathbb{R}, 
$$
where
$$
I=\left( \begin{matrix} {{1}} & {{0}} \\ {{0}} & {{1}} \\ \end{matrix} \right), \; \; X=\left( \begin{matrix} {{0}} & {{1}} \\ {{1}} & {{0}} \\ \end{matrix} \right), \; \; Y=\left( \begin{matrix} {{0}} & {{-i}} \\ {{i}} & {{0}} \\ \end{matrix} \right), \; \; Z=\left( \begin{matrix} {{1}} & {{0}} \\ {{0}} & {{-1}} \\ \end{matrix} \right). 
$$

To express an observable, we use the `SparsePauliOp` class, and as an example, let us consider the following two-qubits observable:

$$
O = I \otimes I + \frac{1}{2} X \otimes X - Y \otimes Y - \frac{1}{4} Z \otimes Z
$$

The first argument corresponds to a list of strings of Pauli terms, and the second argument the corresponding coefficients $\alpha_{k}$.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

obs_ex1 = SparsePauliOp(["II", "XX", "YY", "ZZ"], coeffs=[1, 0.5, -1, -0.25])
print(obs_ex1)
print(obs_ex1.to_matrix())      # Convert the observable to a matrix

As the Pauli operators form a **basis** of the $2^n \times 2^n$ Hermitian matrices, we can represent any operator in terms of Pauli operators with the following expansion:


$$
O=\sum_{P \in\{I, X, Y, Z \}^{\otimes n}} \mathrm{T r} ( O P ) P, 
$$

where the sum runs over all possible $n$-qubit Pauli terms and $\mathrm{T r}$ is the trace of a matrix (sum of diagonal elements). This can also be implemented directly using the `SparsePauliOp.from_operator` method.

In [ ]:
from scipy.linalg import ishermitian

matrix = np.array(
    [[-1, 0, 0.5, -1],
    [0, 1, 1, 0.5],
    [0.5, 1, -1, 0],
    [-1, 0.5, 0, 1]]
)
print(ishermitian(matrix))  # Check first if the matrix is Hermitian

obs_ex2 = SparsePauliOp.from_operator(matrix)
print(obs_ex2)

Let $A$ be an observable and $|\psi \rangle$ a quantum state. The **expectation value** of $A$ in the state $|\psi \rangle$ is denoted as $\langle\psi | A |\psi \rangle,$ and can be understood as the probabilistic expected value of the measurement of an experiment. How do we compute an expectation values in practice?

At the start of the course, we said that to extract the results out of a quantum circuit, we perform a measurement in the Pauli-$Z$ basis (also known as the computational basis), that is, we measures the states $|0\rangle$ or $|1\rangle$. Therefore, a measurement yields $0$ or $1$, depending on the overlap with the states $|0\rangle$ or $|1\rangle$.

$$
| \psi \rangle\xrightarrow{m e a s u r e} \begin{cases} {{0, \mathrm{w i t h ~ p r o b a b i l i t y} \, p_{0}=| \langle \psi | 0 \rangle|^{2},}} \\ {{1, \mathrm{w i t h ~ p r o b a b i l i t y} \, p_{1}=| \langle \psi | 1 \rangle|^{2}.}} \\ \end{cases}
$$

The Pauli $Z$ operator can be rewritten in terms of its eigenvectors (which correspond to the computational basis elements) as follows:

$$ Z = \left( \begin{matrix} {{1}} & {{0}} \\ {{0}} & {{-1}} \\ \end{matrix} \right) = 1 \cdot \left( \begin{matrix} {{1}} & {{0}} \\ {{0}} & {{0}} \\ \end{matrix} \right) + (-1) \cdot \left( \begin{matrix} {{0}} & {{0}} \\ {{0}} & {{1}} \\ \end{matrix} \right) = 1 \cdot |0\rangle \langle 0 | + (-1) \cdot |1\rangle \langle 1|.  $$

Then, we can express the expectation value of $Z$ as

$$ \langle\psi | Z |\psi \rangle = 1 \cdot \langle\psi|0\rangle \langle 0 |\psi \rangle + (-1) \cdot \langle\psi|1\rangle \langle 1|\psi \rangle = 1 \cdot |\langle\psi |0\rangle|^2 + (-1) \cdot |\langle\psi |1\rangle|^2 = 1 \cdot p_0 + (-1) \cdot p_1.$$

Remember that we can only obtain approximation of these probabilities! If we have $N$ shots (that is, if we repeat the experiment $N$ times), we can obtain an approximation of $\langle\psi | Z |\psi \rangle$ given by:

$$ \langle\psi | Z |\psi \rangle = p_0 - p_1 \approx \frac{n_0}{N} - \frac{n_1}{N},$$
where $n_0$ and $n_1$ correspond to the number of times that we measure the state $0$ and $1$, respectively.

We can only measure observables that are diagonal in the computational basis, that is, observables consisting only of $I$ and $Z$ terms. For example, say we want to compute the following expectation value,

$$ \langle + | Z | + \rangle =  1 \cdot |\langle + |0\rangle|^2 + (-1) \cdot |\langle + |1\rangle|^2 \approx \frac{n_0}{N} - \frac{n_1}{N} $$

we use the following circuit. What is the exact expectation value?

In [ ]:
circuit_exp = QuantumCircuit(1,1)
# prepare state |+>

circuit_exp.h(0)

circuit_exp.barrier()

# measure in the Z basis (by default)

circuit_exp.measure(0,0)
circuit_exp.draw("mpl")

In [ ]:
shots = 1000000                                                        # The more shots, the more accurate the approximation

job_sampler2 = sampler.run([(circuit_exp)], shots = shots)           # we can also use the sampler to run a circuit without parameters                   
data_sampler2 = job_sampler2.result()[0].data                        # get data
counts2 = data_sampler2.c.get_counts()                               # get the counts (notice that the classical register is named "c" instead of "meas" this time)
print(counts2)
counts_zero = counts2.get('0', 0)                                    # get the counts for the 0 state   
counts_one= counts2.get('1', 0)                                      # get the counts for the 1 state    

print(f'Exact Expectation Value: {0}')
print(f'Approximate Expectation Value: {(counts_zero-counts_one)/shots}')

Measuring arbitrary Pauli terms therefore requires a change of basis to diagonalize them. To do this, we perform the following transformations,

$$
\begin{array} {l} {{{{X \to Z=H X H}}}} \\ {{{{Y \to Z=H S^{\dagger} Y S H}}}} \\ \end{array} 
$$

where $S=\sqrt{Z}$ corresponds to the phase gate. We can check for the Pauli $X$ operator that the previous transformation holds: 


$$ X = \left( \begin{matrix} {{0}} & {{1}} \\ {{1}} & {{0}} \\ \end{matrix} \right) = 1 \cdot \left[\frac{1}{2} \left( \begin{matrix} {{1}} & {{1}} \\ {{1}} & {{1}} \\ \end{matrix} \right)\right] + (-1) \cdot \left[\frac{1}{2} \left( \begin{matrix} {{1}} & {{-1}} \\ {{-1}} & {{1}} \\ \end{matrix} \right)\right] = 1 \cdot |+\rangle \langle + | + (-1) \cdot |-\rangle \langle -|.  $$

$$ X \to H X H = 1 \cdot H|+\rangle \langle + |H + (-1) \cdot H|-\rangle \langle -|H = 1 \cdot |0\rangle \langle 0 | + (-1) \cdot |1\rangle \langle 1| = Z $$


From the above, if we want to measure the following expectation value $ \langle 0 | X | 0 \rangle$ using a quantum computer, we first rewrite it in terms of $Z$:


$$ \langle 0 | X | 0 \rangle = \langle 0 | H H X H H | 0 \rangle  = \langle + | H X H | + \rangle = \langle + | Z | + \rangle, $$

where we used the fact that $HH = I$ and $H | 0 \rangle = | + \rangle$. We notice that it is equivalent to the circuit that we used before, where we first prepare the state $| 0 \rangle$, and change the basis by adding a Hadamard gate.

In [ ]:
circuit_exp = QuantumCircuit(1,1)
# prepare state |0> (by default)

circuit_exp.barrier()

# measure in the X basis, by adding a Hadamard gate before the measurement

circuit_exp.h(0)

circuit_exp.measure(0,0)
circuit_exp.draw("mpl")

If we are interested in computing the following, more general, expectation value,

$$ \langle \psi(\theta) | ZYX | \psi(\theta) \rangle, \quad | \psi(\theta) \rangle =  (I \otimes I \otimes RX(\theta)) \left[ \frac{1}{\sqrt{2}} \left(| 000 \rangle + | 111 \rangle  \right)\right] $$

we would have to add the corresponding change of basis to the circuit.

In [ ]:
theta = Parameter('θ')     

# create GHZ state
ghz_rx = QuantumCircuit(3)
ghz_rx.h(0)
ghz_rx.cx(0, 1)
ghz_rx.cx(1, 2)

# Apply RX rotation to the first qubit
ghz_rx.rx(theta, 0)


ghz_rx_exp = ghz_rx.copy()  # copy the circuit for later use
ghz_rx.barrier()

# measure in the X basis, adding the Hadamard gate
ghz_rx.h(0)
 
# measure in the Y, adding a Hadamard and S^\dagger gates
ghz_rx.sdg(1)
ghz_rx.h(1)
 
# the Z basis is the default, no action required here
 
ghz_rx.barrier() 
ghz_rx.draw("mpl")

The reference implementation of `EstimatorV2` in `qiskit.primitives` that runs on a local statevector simulators is the `StatevectorEstimator` class. It takes circuits, observables, and parameters as inputs and returns the locally computed expectation values.

In [ ]:
obs_ex3 = SparsePauliOp(["XYZ"], coeffs=[1])                # observable(s) whose expected values you want to compute
 
parameter_values = [[0], [np.pi/4], [np.pi/2]]              # value(s) for the circuit parameter(s)

Notice that we run the `Estimator` with a single circuit and a single observable, but with three parameter values. Therefore, we will get three expectation values, one for each parameter.

In [ ]:
ghz_rx_exp.draw("mpl")

In [ ]:
from qiskit.primitives import StatevectorEstimator
estimator = StatevectorEstimator()

job_estimator = estimator.run([(ghz_rx_exp, obs_ex3, parameter_values)])
result_estimator = job_estimator.result()[0]
data_estimator = result_estimator.data       

print(f"Expectation value: {data_estimator.evs}")

## Primitive Unified Bloc (PUB)

The **Primitive Unified Bloc** (pub) programs are designed to handle parametric quantum circuits, and correspond to the **inputs** for the `Sampler` and `Estimator` primitives.

For the `Sampler` primitive, the format of the pub contains three values:

* A parametric `QuantumCircuit`, which may contain one or more `Parameter` objects. These circuits should also **include measurement instructions** and **classical registers** for each of the qubits to be sampled.
* A collection of parameter values to bind the circuit against, if the `Parameter` objects must be bound at runtime.
* (Optionally) a number of shots to measure the circuit with.

For the `Estimator` primitive, the format of the pub contains four values:

* A parametric `QuantumCircuit`, which contain one or more `Parameter` objects.
* A list of one or more observables (`ObservablesArrayLike`), which specify the expectation values to estimate, arranged into an array (for example, a single observable represented as a 0-d array, a list of observables as a 1-d array). Examples of `ObservablesArrayLike` include `Pauli`, `SparsePauliOp`, `PauliList`, or a string.
* A collection of parameter values to bind the circuit against. This can be specified as a single array-like object where the last index is over circuit `Parameter` objects.
* (Optionally) a target precision for expectation values to estimate.

**Observation 1**: We can also consider non-parametric `QuantumCircuit`s, in which case the parameter values are set to `None`.

**Observation 2**: We can provide a `list` of pubs to the primitives, executing them in batches.

**Observation 3**: The pubs aggregate elements from multiple arrays (observables and parameter values) by following the same [broadcasting rules](https://docs.quantum.ibm.com/guides/primitive-input-output#broadcasting-rules) as NumPy (expansion of arrays with different shapes during arithmetic operations so they behave as if they had the same shape).

## Sampler Outputs

Upon running a Sampler Job, we will get a `PrimitiveResult` object which contains a list of `PubResult`'s corresponding to each of the input pubs. To extract information out of our `PubResult`, we can use the following methods:

* `PubResult.data`: Contains the **measurement outcome data** for all classical registers in the input pubs circuit.
* `PubResult.metadata`: Contains any additional metadata that a primitive might record.

The `PubResult.data` method will return container called a `DataBin`, which stores the outcomes of measurements in the `ClassicalRegister` in the pub circuit. Each register's data is stored in a `BitArray` container.

In [ ]:
qc_param.draw("mpl")

In [ ]:
sampler = StatevectorSampler()

params = np.array([0, np.pi/2])                   # Define (two) parameters for the circuit
pubs = (qc_param, params)                         # Define (one) pub for Sampler

job_sampler_s = sampler.run([pubs], shots = 20)   # We have two parameters: we get one PubResult with a BitArray of size 2
print(type(job_sampler_s))

print('----------------------------------------') 

primitive_result_s = job_sampler_s.result()
print(primitive_result_s)

print('----------------------------------------')

pub_result_s = primitive_result_s[0]           # Get the first (only) PubResult from the PrimitiveResult
print(pub_result_s)

print('----------------------------------------')

data_pub_s = pub_result_s.data                # Get DataBin
print(data_pub_s)

print('----------------------------------------')


metadata_pub_s = pub_result_s.metadata        # Get Metadata
print(metadata_pub_s)

print('----------------------------------------')


bits = data_pub_s.meas                        # Get the bits from the DataBin (notice that the classical register is named "meas") 
print(bits)

The `BitArray` class stores the single-shot outcomes of all classical registers. In general, we are mostly interested in the total counts to get the probability distributions, which is accessed through the `bits.get_counts()` method, similar to before. Nonetheless, we can also access the value of each shot through the `bits.array` attribute.

In [ ]:
bits[0].get_counts()        # get the counts for the first parameter

In [ ]:
bits[1].get_counts()        # get the counts for the second parameter

In [ ]:
bits[1].array               # get the value of each shot from the second parameter

The number of shots can be specified inside of each pub, or as a default argument for the run method. The pub parameters take priority over the second one.

In [ ]:
pub1 = (qc_param, 0, 100)   # 100 shots
pub2 = (qc_param, np.pi/2)

result1 = sampler.run([pub1, pub2], shots = 500).result()

for i, res in enumerate(result1):
    print(f'Pub {i}: num_shots = {res.data.meas.num_shots}')

## Estimator Outputs

In [ ]:
ghz_rx_exp.draw("mpl")

Similar to the Sampler, an Estimator Job will contain a `PrimitiveResult` with a list of `PubResults` for each input pub. Instead of measure outcomes, its `DataBin` contains the following fields:

* `evs`: The mean expectation value estimates for all input pub parameter values and observables.
* `stds`: The standard deviations of the mean of expectation value estimates. (for the `StatevectorEstimator()` it is always zero).

In the Sampler case, we were able to modify the amount of shots. Here, we can modify the **precision argument**, which expresses the error bars that the primitive implementation should target for expectation values estimates. This is the sampling overhead (also known as shot noise) and is defined exclusively in the `estimator.run()` method.

In [ ]:
estimator = StatevectorEstimator()

obs_ex4 = [SparsePauliOp(["XYZ"], coeffs=[0.5])]
obs_ex5 = [SparsePauliOp(["III", "XXX", "YYY", "ZZZ"], coeffs=[1, 0.5, -1, -0.25])]

parameter_values4 = [[0], [np.pi/3], [np.pi/5]]
parameter_values5 = [[0], [np.pi/4], [np.pi/6]]

job_e = estimator.run([(ghz_rx_exp, [obs_ex4, obs_ex5], [parameter_values4, parameter_values5])], precision=0.0)        # Two observables, with two parameter values each
print(type(job_e))

print('----------------------------------------')

primitive_result_e = job_e.result()
print(primitive_result_e)                              

print('----------------------------------------') 


pub_result_e = primitive_result_e[0]                     # Get the first (only) PubResult from the PrimitiveResult
print(pub_result_e)
print(pub_result_e.metadata)                             # Metadata for the PubResult
print('----------------------------------------') 

data_pub_e = pub_result_e.data
print(data_pub_e)                                        # Our DataBin has shape (2,3) as we have 2 observables and 3 parameter values

print('----------------------------------------') 

print(data_pub_e.evs)   # The expectation values for each parameter value
print(data_pub_e.stds)  # The standard deviations for each parameter value

## Qiksit Aer Primitives

The previous examples used the reference primitives in the Qiskit SDK `StatevectorSampler` and `StatevectorEstimator`, which are implemented with full state vector simulation, that is, in a noiseless scenario. If we want to simulate larger circuits, include noise models or use FakeBackends, then we have to use the primitives as defined in Qiskit Aer.

In [ ]:
from qiskit.circuit.library import EfficientSU2     # qiskit.library has many predefined circuits, we will study some later on

n_qubits = 5
qc_su2 = EfficientSU2(n_qubits)
qc_su2.decompose().draw("mpl")

We import the `EstimatorV2` and `SamplerV2` primitives from `qiskit_aer.primitives`. Let us calculate the expectation value of the previous circuit with the observable

$$O = XXXXX + \frac{1}{2} XYXYX - ZZZZZ - \frac{1}{4}ZXZXZ.$$

This primitives require circuits and observables to be transformed to only use instructions supported by the `Backend`, referred to as **instruction set architecture** (ISA) circuits and observables.

* An ISA circuit is a `QuantumCircuit` satisfying:
    * The same number of qubits as the backend.
    * Only contains gates from the basis gates of the backend.
    * Satisfies the connectivity of the backend.

An ISA circuit is typically obtained via transpilation.

* An ISA observable is an observable such that
    * It corresponds to a linear combination of Pauli operators with real coefficients.
    * Each Pauli operator is defined on the same number of qubits as an ISA circuit.

For a given observable, we need to apply the layout of a transpiled circuit to map the observable qubits. For `SparsePauliOp` operator classes, this can be done through the `observable.apply_layout()` method (this will become relevant when discussing `qiskit_ibm_runtime.primitives`).

In [ ]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import EstimatorV2 as EstimatorAer       # Choose a different name to avoid confusion with the previous Estimator class

observable = SparsePauliOp(["X"*n_qubits, "XYXYX", "Z"*n_qubits, "ZXZXZ"], coeffs=[1, 0.5, -1, -0.25])               # We can multiply a string with a number -> "X"*n_qubits = "XXXXX"

num_params = qc_su2.num_parameters                  # We can get the number of parameters from the circuit using this method
np.random.seed(0)                                   # We set the seed to get reproducible results
params = 2*np.pi*np.random.rand(num_params)         # We generate random values for the parameters

backend = AerSimulator()
exact_estimator = EstimatorAer()

pm = generate_preset_pass_manager(backend=backend)
qc_trans = pm.run(qc_su2)                           # Transpile to get a ISA circuit

pub = (qc_trans, observable, params)                # We define the pub
job = exact_estimator.run([pub], precision=0.0)
result = job.result()
pub_result = result[0]
exact_value = float(pub_result.data.evs)
std = float(pub_result.data.stds)

print(f'Exact Expectation Value: {exact_value}')
print(f'Standard Deviation: {std}')

Let us consider now a `NoiseModel` which adds a depolarizing error of $1\%$ to each $CNOT$ gate. In practice, the error arising from the two-qubit gates are the dominant source of error when running a circuit. We can include this `NoiseModel` directly into our Estimator through the `options` parameter (which will be studied further later).

In [ ]:
from qiskit_aer.noise import NoiseModel, depolarizing_error

noise_model = NoiseModel()

cx_depolarizing_prob = 0.01
dep_error = depolarizing_error(cx_depolarizing_prob, 2)
noise_model.add_all_qubit_quantum_error(dep_error, ["cx"])

options = dict(backend_options=dict(noise_model=noise_model))
print(options)

In [ ]:
noisy_estimator = EstimatorAer(options=options)

job = noisy_estimator.run([pub], precision=0.0)       # The standard deviation will be equivalent to the precision for the Aer Estimator
result = job.result()
pub_result = result[0]
noisy_value = float(pub_result.data.evs)
std = float(pub_result.data.stds)

print(f'Exact Expectation Value: {exact_value}')
print(f'Noisy Expectation Value: {noisy_value}')
print(f'Standard Deviation: {std}')
print(f'Absolute Error: {np.abs(noisy_value-exact_value)}')

We can add error to the Sampler in a similar manner. Lets compare the measurements outputs for a GHZ state of $5$ qubits, with the same depolarizing error on each $CNOT$.

$$| GHZ_5 \rangle = \frac{1}{\sqrt{2}} \left(| 00000 \rangle + | 11111 \rangle  \right)$$

In [ ]:
from qiskit_aer.primitives import SamplerV2 as SamplerAer

shots = 10000
qc_ghz5 = QuantumCircuit(n_qubits)
qc_ghz5.h(0)
qc_ghz5.cx(0, 1)
qc_ghz5.cx(1, 2)
qc_ghz5.cx(2, 3)
qc_ghz5.cx(3, 4)
qc_ghz5.measure_all()
qc_ghz5.draw("mpl")

In [ ]:
pm = generate_preset_pass_manager(backend=backend)
qc_trans = pm.run(qc_ghz5) 

pub = (qc_trans, [], shots)                 # we define the pub, without parameters

sampler = SamplerAer()
job = sampler.run([pub])
result = job.result()
pub_result = result[0]
counts = pub_result.data.meas.get_counts()

In [ ]:
noisy_sampler = SamplerAer(options=options)     # Adds the previously defined depolarizing noise

pub = (qc_trans, [], shots)
job = noisy_sampler.run([pub])
result = job.result()
pub_result = result[0]
counts_noise = pub_result.data.meas.get_counts()

In [ ]:
from qiskit.visualization import plot_histogram

legend = ['Counts', 'Noisy Counts']
plot_histogram([counts, counts_noise], legend=legend, sort='value_desc', number_to_keep=10)